In [1]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,BatchNormalization,Dropout,MaxPool2D,Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import cross_val_score# imports the cross val score which is simply used to make cross validation to
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier#imports the keras classifier class that is used to read the model
import glob
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
E:\conda\envs\iris\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\conda\envs\iris\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\conda\envs\iris\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\conda\envs\iris\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Pas

In [2]:
model=Sequential()
model.add(Conv2D(64,3,3,activation='relu',input_shape=(256,256,3)))
model.add(Conv2D(54,3,3,activation='relu'))    
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(36,3,3,activation='relu'))    
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(16,3,3,activation='relu'))    
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

E:\conda\envs\iris\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", input_shape=(256, 256,...)`
  


Instructions for updating:
Colocations handled automatically by placer.


E:\conda\envs\iris\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(54, (3, 3), activation="relu")`
  This is separate from the ipykernel package so we can avoid doing imports until


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


E:\conda\envs\iris\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (3, 3), activation="relu")`
  import sys
E:\conda\envs\iris\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu")`
  # This is added back by InteractiveShellApp.init_path()


In [3]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 252, 54)      31158     
_________________________________________________________________
batch_normalization_1 (Batch (None, 252, 252, 54)      216       
_________________________________________________________________
dropout_1 (Dropout)          (None, 252, 252, 54)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 54)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 124, 124, 36)      17532     
_________________________________________________________________
batch_normalization_2 (Batch (None, 124, 124, 36)      144       
__________

In [4]:
import visualkeras
visualkeras.layered_view(model, to_file='model.png') # write to disk
visualkeras.layered_view(model).show() # display using your system viewer


In [4]:
infected_dirs=glob.glob(r'data//COVID//*')
normal_dirs=glob.glob(r'data//non-COVID//*')
my_list=[normal_dirs,infected_dirs]
X=[]
y=[]
for index_dirs,dirs in enumerate(my_list):
    for img_dir in dirs:
        img=cv2.imread(img_dir)
        img=cv2.resize(img,(256,256))
        img=np.array(img)
        X.append(img)
        y.append(index_dirs)
        


In [5]:
len(y)

2690

In [6]:
np_x=np.array(X)
np_y=np.array(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(np_x, np_y, test_size=0.2, random_state=10)

In [8]:
history=model.fit(X_train,y_train,batch_size=10,epochs=5,validation_data=(X_test,y_test))

Instructions for updating:
Use tf.cast instead.
Train on 2152 samples, validate on 538 samples
Epoch 1/5
2152/2152 [==============================] - 1621s 753ms/step - loss: 0.6583 - acc: 0.7277 - val_loss: 1.0877 - val_acc: 0.6413
Epoch 2/5
2152/2152 [==============================] - 1440s 669ms/step - loss: 0.3946 - acc: 0.8257 - val_loss: 1.1920 - val_acc: 0.6245
Epoch 3/5
2152/2152 [==============================] - 1670s 776ms/step - loss: 0.2761 - acc: 0.8885 - val_loss: 0.5132 - val_acc: 0.7268
Epoch 4/5
2152/2152 [==============================] - 1976s 918ms/step - loss: 0.2365 - acc: 0.9075 - val_loss: 0.8128 - val_acc: 0.7788
Epoch 5/5
2152/2152 [==============================] - 1570s 730ms/step - loss: 0.1943 - acc: 0.9219 - val_loss: 1.3479 - val_acc: 0.6115


In [9]:
model.save('classfier_binary_4.h5')

In [35]:
pred=model.predict(X_test)

In [36]:
final=[]
for i in pred:
    if i>=0.5:
        final.append(1)
    if i<0.5:
        final.append(0)

In [37]:
final=np.array(final)

In [38]:
final

array([1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,

In [39]:
# we will implement it manually because we cant genrate multi label confusion metrics from keras
tp=0
fp=0
fn=0
tn=0
right=0
for i in range(len(final)):
    t=y_test[i]
    p=final[i]
    if p==t:
        right+=1
    if p==0 and t==0:
        tn+=1
    if p==0 and t==1:
        fn+=1
    if p==1 and t==0:
        fp+=1
    if p==1 and t==1:
        tp+=1

precesion=tp/(tp+fp)
recall=tp/(tp+fn)
print(f'precsion valueis = {precesion}')
print(f'recall value is = {recall}')
print(f'accurcy is = {right/len(final)}')
print("\n\n")
confusion_matrix=np.array([tp,tn,fp,fn])
print("confusion_matrix ="+"\n"+str(confusion_matrix))


precsion valueis = 0.9291044776119403
recall value is = 0.8383838383838383
accurcy is = 0.8754646840148699



confusion_matrix =
[249 222  19  48]


# train precesion and recall